In [ ]:
"""
Map each description to vector using word2vec.
"""

import os
import csv
import random
import gensim
import numpy as np

word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
print("Loaded word vectors successfully!")

def doc_to_vec(words, word2vec):
    # get list of word vectors in sentence
    word_vecs = [word2vec.get_vector(w) for w in words if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

In [ ]:
import pickle
import os.path
from sklearn.decomposition import PCA

train_description_feature_map = pickle.load(open('./../features/12_4_[train]_description_feature_map_py27.pkl', 'rb'))
test_derscription_feature_map = pickle.load(open('./../features/12_4_[test]_description_feature_map_py27.pkl', 'rb'))
print "loaded description map..."

In [3]:
train_img_map = pickle.load(open('./../features/12_1_train_img_feature_map_py27.pkl', 'rb'))
test_img_map = pickle.load(open('./../features/12_1_test_img_feature_map_py27.pkl', 'rb'))
print "loaded img feature map..."

train_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], train_img_map)))
test_pool5_img = np.asarray(list(map(lambda x: x["POOL_vector"], test_img_map)))

loaded description map...
loaded img feature map...


In [17]:
print train_description_feature_map[0]

{u'description_complete': u'The skateboarder is putting on a show using the picnic table as his stage. A skateboarder pulling tricks on top of a picnic table. A man riding on a skateboard on top of a table. A skate boarder doing a trick on a picnic table. A person is riding a skateboard on a picnic table with a crowd watching.', u'name': u'0.txt', u'description_n_v_adj': u'skateboarder show picnic table stage. skateboarder top picnic table. man skateboard top table. skate boarder trick picnic table. person skateboard picnic table crowd watching.', 'doc_vec': array([ 0.03252411, -0.04455058,  0.04320781,  0.1703542 , -0.04508464,
       -0.0599645 ,  0.00985718, -0.03812663,  0.02850978,  0.01871967,
        0.10940424, -0.10728455,  0.08560181, -0.04368083, -0.08532715,
        0.07139079, -0.04007339,  0.0111618 ,  0.04828644, -0.0137736 ,
        0.12915039,  0.17485714, -0.09606425, -0.04476166, -0.01881409,
       -0.00075277, -0.09360758,  0.13049825,  0.05255508, -0.07037608,
   

In [ ]:
"""
PLSR
"""
model_name = "./../models/pls_800_pool5_to_word2vec_all_300.pkl"
if os.path.exists(model_name):
    model = pickle.load(open(model_name, 'rb'))
else:
    # train PLSR.
    model = PLSRegression(n_components=800)
    model.fit(train_pool5_img, des_BOW_all_train_pca)
    pickle.dump(model, open(model_name, 'wb'), protocol=2)
print "model loaded..."